In [0]:
from azure.storage.blob import BlobServiceClient
import io
import pandas as pd

connection_string = "DefaultEndpointsProtocol=https;AccountName=thirdtry3;AccountKey=R2tz5fvj0ZdyJ2Cjb4OjyRlaNc1MS9tTPfc5uWjWjnpzm2C7MWtZr1k+7Hnawhesoj/eIPfO05+F+AStoGzFjQ==;EndpointSuffix=core.windows.net"

container_name = "mycontainer"
blob_name = "product_dataset.csv"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

blob_data = blob_client.download_blob()

csv_data = blob_data.readall()

data_frame = pd.read_csv(io.BytesIO(csv_data))

print(data_frame)

   product_id     product_name  ... customer_location  target_column
0        2001           Tablet  ...          Illinois              1
1        2002        Microwave  ...           Georgia              0
2        2003             Desk  ...        Washington              1
3        2004          Monitor  ...           Florida              1
4        2005  Air Conditioner  ...           Arizona              1
5        2006        Bookshelf  ...            Oregon              0
6        2007          Toaster  ...              Ohio              1
7        2008          Printer  ...          Michigan              0
8        2009   Vacuum Cleaner  ...             Texas              1
9        2010       Smartwatch  ...        California              1

[10 rows x 9 columns]


In [0]:
import pandas as pd
import numpy as np

data_frame['cumulative_quantity_sold'] = data_frame['quantity_sold'].cumsum()

data_frame['rolling_avg_price'] = data_frame['price'].rolling(window=3).mean()
data_frame['rolling_avg_quantity_sold'] = data_frame['quantity_sold'].rolling(window=3).mean()
data_frame['rolling_avg_discount'] = data_frame['discount'].rolling(window=3).mean()

if 'date' in data_frame.columns:
    data_frame['year'] = data_frame['date'].dt.year
    data_frame['month'] = data_frame['date'].dt.month
    data_frame['day'] = data_frame['date'].dt.day
    data_frame['weekday'] = data_frame['date'].dt.weekday

data_frame = pd.get_dummies(data_frame, columns=['category', 'customer_location'], drop_first=True)

data_frame['price_quantity_interaction'] = data_frame['price'] * data_frame['quantity_sold']

if data_frame['price'].skew() > 1:
    data_frame['log_price'] = np.log1p(data_frame['price'])

if data_frame['quantity_sold'].skew() > 1:
    data_frame['log_quantity_sold'] = np.log1p(data_frame['quantity_sold'])

data_frame['price_per_unit'] = data_frame['price'] / (data_frame['quantity_sold'] + 1)

data_frame = data_frame.drop(columns=['product_name', 'target_column'])

print(data_frame)

   product_id  price  ...  log_quantity_sold  price_per_unit
0        2001    500  ...           2.944439       26.315789
1        2002    200  ...           2.708050       13.333333
2        2003    250  ...           2.397895       22.727273
3        2004    180  ...           3.258097        6.923077
4        2005   1200  ...           2.079442      150.000000
5        2006    150  ...           2.833213        8.823529
6        2007     40  ...           3.931826        0.784314
7        2008    250  ...           2.564949       19.230769
8        2009    300  ...           3.044522       14.285714
9        2010    220  ...           3.433987        7.096774

[10 rows x 25 columns]


In [0]:
%python
from azure.storage.blob import BlobServiceClient
import io
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

connection_string = "DefaultEndpointsProtocol=https;AccountName=thirdtry3;AccountKey=R2tz5fvj0ZdyJ2Cjb4OjyRlaNc1MS9tTPfc5uWjWjnpzm2C7MWtZr1k+7Hnawhesoj/eIPfO05+F+AStoGzFjQ==;EndpointSuffix=core.windows.net"

container_name = "mycontainer"
blob_name = "product_dataset.csv"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

blob_data = blob_client.download_blob()

csv_data = blob_data.readall()

data_frame = pd.read_csv(io.BytesIO(csv_data))

print(data_frame)

data_frame['cumulative_quantity_sold'] = data_frame['quantity_sold'].cumsum()

data_frame['rolling_avg_price'] = data_frame['price'].rolling(window=3).mean()
data_frame['rolling_avg_quantity_sold'] = data_frame['quantity_sold'].rolling(window=3).mean()
data_frame['rolling_avg_discount'] = data_frame['discount'].rolling(window=3).mean()

data_frame = pd.get_dummies(data_frame, columns=['category', 'customer_location'], drop_first=True)

data_frame['price_quantity_interaction'] = data_frame['price'] * data_frame['quantity_sold']

if data_frame['price'].skew() > 1:
    data_frame['log_price'] = np.log1p(data_frame['price'])

if data_frame['quantity_sold'].skew() > 1:
    data_frame['log_quantity_sold'] = np.log1p(data_frame['quantity_sold'])

data_frame['price_per_unit'] = data_frame['price'] / (data_frame['quantity_sold'] + 1)

columns_to_drop = ['product_name', 'target_column']
data_frame = data_frame.drop(columns=[col for col in columns_to_drop if col in data_frame.columns])

if 'target_column' in data_frame.columns:
    X = data_frame.drop(columns=['target_column'])
    y = data_frame['target_column']

    X = X.dropna()
    y = y[X.index]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(X_train)
else:
    print("The 'target_column' does not exist in the DataFrame.")

   product_id     product_name  ... customer_location  target_column
0        2001           Tablet  ...          Illinois              1
1        2002        Microwave  ...           Georgia              0
2        2003             Desk  ...        Washington              1
3        2004          Monitor  ...           Florida              1
4        2005  Air Conditioner  ...           Arizona              1
5        2006        Bookshelf  ...            Oregon              0
6        2007          Toaster  ...              Ohio              1
7        2008          Printer  ...          Michigan              0
8        2009   Vacuum Cleaner  ...             Texas              1
9        2010       Smartwatch  ...        California              1

[10 rows x 9 columns]
The 'target_column' does not exist in the DataFrame.


In [0]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

data = {
    'Feature1': [1, 2, np.nan, 4, 5],
    'Feature2': [5, 6, 7, 8, 9],
    'Price': [100, 200, 300, 400, 500]
}
df = pd.DataFrame(data)

X = df.drop(columns=['Price'])
y = df['Price']

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared (R²): {r2}")


Mean Absolute Error (MAE): 1.1368683772161603e-13
Mean Squared Error (MSE): 1.2924697071141057e-26
Root Mean Squared Error (RMSE): 1.1368683772161603e-13
R-squared (R²): nan


/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [0]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared (R²): {r2}")

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_rmse_scores = cross_val_score(model, X_imputed, y, cv=kf, scoring='neg_root_mean_squared_error')
cv_r2_scores = cross_val_score(model, X_imputed, y, cv=kf, scoring='r2')

cv_rmse_scores = -cv_rmse_scores

print(f"Cross-Validation RMSE Scores: {cv_rmse_scores}")
print(f"Mean CV RMSE: {cv_rmse_scores.mean()}")

print(f"Cross-Validation R² Scores: {cv_r2_scores}")
print(f"Mean CV R²: {cv_r2_scores.mean()}")

Mean Absolute Error (MAE): 1.1368683772161603e-13
Mean Squared Error (MSE): 1.2924697071141057e-26
Root Mean Squared Error (RMSE): 1.1368683772161603e-13
R-squared (R²): nan
Cross-Validation RMSE Scores: [0.00000000e+00 1.70530257e-13 0.00000000e+00 1.13686838e-13
 0.00000000e+00]
Mean CV RMSE: 5.684341886080802e-14
Cross-Validation R² Scores: [nan nan nan nan nan]
Mean CV R²: nan


/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/databricks/python/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two

In [0]:
import joblib
from azure.storage.blob import BlobServiceClient
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = np.random.rand(100, 5)
y = np.random.rand(100)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

model_filename = "linear_regression_model.pkl"
joblib.dump(model, model_filename)

connection_string = "DefaultEndpointsProtocol=https;AccountName=thirdtry3;AccountKey=R2tz5fvj0ZdyJ2Cjb4OjyRlaNc1MS9tTPfc5uWjWjnpzm2C7MWtZr1k+7Hnawhesoj/eIPfO05+F+AStoGzFjQ==;EndpointSuffix=core.windows.net"
container_name = "mycontainer"
model_blob_name = "linear_regression_model.pkl"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)

blob_client = blob_service_client.get_blob_client(container=container_name, blob=model_blob_name)

try:
    with open(model_filename, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)  # overwrite=True to replace any existing file

    print(f"Model successfully uploaded to Azure Blob Storage as {model_blob_name}")

    os.remove(model_filename)

except Exception as e:
    print(f"Error uploading model to Azure Blob Storage: {e}")

Mean Squared Error (MSE): 0.06946967833526876
Model successfully uploaded to Azure Blob Storage as linear_regression_model.pkl
